In [ ]:
import requests
from bs4 import BeautifulSoup
import json, pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
keyword = "부천시 옥길동"

headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
url = "https://m.land.naver.com/search/result/{}".format(keyword)
res = requests.get(url, headers=headers)
res.raise_for_status()

soup = (str)(BeautifulSoup(res.text, "lxml"))

In [ ]:
value = soup.split("filter: {")[1].split("}")[0].replace(" ","").replace("'","")


In [ ]:

lat = value.split("lat:")[1].split(",")[0]
lon = value.split("lon:")[1].split(",")[0]
z = value.split("z:")[1].split(",")[0]
cortarNo = value.split("cortarNo:")[1].split(",")[0]
rletTpCds = value.split("rletTpCds:")[1].split(",")[0]
tradTpCds = value.split("tradTpCds:")[1].split()[0]

# lat - btm : 37.550985 - 37.4331698 = 0.1178152
# top - lat : 37.6686142 - 37.550985 = 0.1176292
lat_margin = 0.118

# lon - lft : 126.849534 - 126.7389841 = 0.1105499
# rgt - lon : 126.9600839 - 126.849534 = 0.1105499
lon_margin = 0.111

btm=float(lat)-lat_margin
lft=float(lon)-lon_margin
top=float(lat)+lat_margin
rgt=float(lon)+lon_margin

# 최초 요청 시 디폴트 값으로 설정되어 있으나, 원하는 값으로 구성
rletTpCds="APT" #상가
tradTpCds="A1:B1:B2" #매매/전세/월세 매물 확인

In [ ]:
# clusterList?view 를 통한 그룹(단지)의 데이터를 가져온다.
# https://m.land.naver.com/cluster/clusterList?view=atcl&cortarNo=1120011300&rletTpCd=APT%3AABYG%3AJGC&tradTpCd=A1%3AB1%3AB2&z=14&lat=37.5419&lon=127.016&btm=37.5139238&lft=126.9682352&top=37.5698657&rgt=127.0637648&pCortarNo=&addon=COMPLEX&isOnlyIsale=false
remaked_URL = "https://m.land.naver.com/cluster/clusterList?view=atcl&cortarNo={}&rletTpCd={}&tradTpCd={}&z={}&lat={}&lon={}&btm={}&lft={}&top={}&rgt={}"\
     .format(cortarNo, rletTpCds, tradTpCds, z, lat, lon,btm,lft,top,rgt)
     
print(remaked_URL)

res2 = requests.get(remaked_URL, headers=headers)
json_str = json.loads(json.dumps(res2.json()))

In [ ]:
print(json_str)

In [ ]:
req_URL = "https://m.land.naver.com/complex/getComplexArticleList?hscpNo=567&tradTpCd=A1&order=point_&showR0=N&page=1"
res = requests.post(req_URL, headers=headers)
json_str = json.loads(json.dumps(res.json()))


APTlist = json_str['result']['list']
df = pd.DataFrame(APTlist)
display(df)

df.to_csv('haha.csv', encoding='utf-8-sig')

In [ ]:
URL = "https://m.land.naver.com/complex/getComplexArticleList"
param = {
    'hscpNo': '567',
    'tradTpCd': 'A1:B1:B2',
    'order': 'point',
    'showR0': 'N',
    'page': '1',

}
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.220 Whale/1.3.51.7 Safari/537.36',
    'Referer': 'https://m.land.naver.com/'
}

req_URL = "https://m.land.naver.com/complex/getComplexArticleList"

resp = requests.post(req_URL, params=param, headers=header)
json_str = json.loads(json.dumps(resp.json()))

APTlist = json_str['result']['list']
df = pd.DataFrame(APTlist)

page = 1
while True:
    page += 1
    param['page'] = page

    req_URL = "https://m.land.naver.com/complex/getComplexArticleList"

    resp = requests.post(req_URL, params=param, headers=header)
    json_str = json.loads(json.dumps(resp.json()))


    if len(json_str['result']['list']) == 0:
         break

    APTlist = json_str['result']['list']
    nex_df = pd.DataFrame(APTlist)
    df = pd.concat([df,nex_df])

display(df)